In [3]:
import os

import numpy as np
import polars as pl
import statsmodels.api as sm
from tqdm import tqdm

In [14]:
data_dir_path = os.path.join('..', '..', "data", 'topic_model_videos')

video_df = pl.DataFrame()
video_dir_path = os.path.join('..', '..', 'data', 'results', '2024_04_10', 'hours', '19')
video_pbar = tqdm(total=60*60, desc='Reading videos')
limit = 1
for root, dirs, files in os.walk(video_dir_path):
    for file in files:
        if file == 'videos.parquet.zstd':
            video_pbar.update(1)
            result_path = os.path.join(root, file)
            batch_video_df = pl.read_parquet(result_path)
            batch_video_df = batch_video_df.select([
                pl.col('video_id'),
                pl.col('authorVerified'),
                pl.col('musicOriginal'),
                pl.col('videoDuration'),
                pl.col('videoQuality'),
                pl.col('locationCreated'),
                pl.col('desc'),
                pl.col('shareCount'),
                pl.col('diggCount'),
                pl.col('commentCount'),
                pl.col('playCount'),
                pl.col('diversificationLabels')
            ])
            video_df = pl.concat([video_df, batch_video_df], how='diagonal_relaxed')
    #         limit += 1
    # if limit > 10:
    #     break

video_topic_df = pl.read_parquet(os.path.join(data_dir_path, "video_topics.parquet.gzip"))
video_topic_df = video_topic_df.with_columns(pl.col('image_path').str.split('/').list.get(-1).str.split('.').list.get(0).alias('video_id'))

topic_info_df = pl.read_parquet(os.path.join(data_dir_path, "topic_info.parquet.gzip"))
topic_name_mapper = {row['Topic']: row['Desc'] for row in topic_info_df.to_dicts()}

video_df = video_df.join(video_topic_df, on='video_id', how='left')

Reading videos: 100%|██████████| 3600/3600 [01:47<00:00, 20.34it/s]

Reading videos: 100%|██████████| 3600/3600 [02:00<00:00, 20.34it/s]

In [18]:
try:
    video_df = video_df.with_columns(pl.col('topic').replace_strict(topic_name_mapper))
except:
    video_df = video_df.with_columns(pl.when(pl.col('topic').is_in(topic_name_mapper.keys())).then(pl.col('topic').replace_strict(topic_name_mapper)).otherwise('No Topic').alias('topic'))

video_df = video_df.with_columns(pl.col('desc').str.extract_all('#[a-zA-Z0-9_]+').alias('hashtags'))
hashtag_df = video_df.select('hashtags').explode('hashtags').filter(pl.col('hashtags').is_not_null())['hashtags'].value_counts().sort('count').filter(pl.col('count') > 50)

video_df = pl.concat([video_df, video_df['topic'].to_dummies(), video_df['locationCreated'].to_dummies()], how='horizontal')

ColumnNotFoundError: No Topic

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["video_id", "authorVerified", "musicOriginal", "videoDuration"]; PROJECT */318 COLUMNS; SELECTION: None

In [22]:
video_df.columns

['video_id',
 'authorVerified',
 'musicOriginal',
 'videoDuration',
 'videoQuality',
 'locationCreated',
 'desc',
 'shareCount',
 'diggCount',
 'commentCount',
 'playCount',
 'diversificationLabels',
 'desc_right',
 'image_path',
 'topic',
 'hashtags',
 'topic_Acoustic Guitar Performances',
 'topic_Agricultural Vehicles And Equipment',
 'topic_Airplane Flight Footage',
 'topic_Animated Character Outfits And Scenes',
 'topic_Birthday Cake Celebration Photo',
 'topic_Branding Elements On Backgrounds',
 'topic_Casual Outdoor Gathering With Beverages',
 'topic_Celestial Depiction Of Divine Figure',
 'topic_Close-Up Selfie Attire And Lighting',
 'topic_Colorful Nail Art Designs And Textures',
 'topic_Construction Footage Analysis',
 'topic_Cozy Indoor Pet Imagery',
 'topic_Culinary Dining Scene',
 'topic_Emotional Expressions In Portrait Images',
 'topic_Equestrian Outdoor Riding Scenes',
 'topic_Fashionable Urban Evening Pedestrian Attire',
 'topic_Garage Automotive Repair Video',
 'topic_

In [27]:
indep_vars = [col for col in video_df.columns if col not in ['video_id', 'hashtags', 'topic', 'locationCreated', 'desc', 'desc_right', 'diversificationLabels', 'image_path', 'shareCount', 'diggCount', 'commentCount', 'playCount']]
dep_vars = ['commentCount', 'diggCount', 'shareCount', 'playCount']
X = video_df[indep_vars]
X = sm.add_constant(X)

TypeError: '>=' not supported between instances of 'bool' and 'NoneType'